The criterion for the mapping is to minimize stress
- Start with a (random) configuration of points in small # dimension
- Apply some form of steepest descent iteratively to minimize the stress
  - move objects
  - If moving objects does not help add dimensions

$$ stress = \sqrt{\frac{\sum_{i,j} (d^{'}_{ij} - d_{ij})^{2}}{\sum_{i,j} d_{ij}^{2}}} $$

$$ {stress}^2 = \frac{\sum_{i,j} (d^{'}_{ij} - d_{ij})^{2}}{\sum_{i,j} d_{ij}^{2}} $$

In [1]:
import sys, os
root = os.path.dirname(os.path.abspath(''))
sys.path.append(root)

In [ ]:
import storage
from util import Constants
db = storage.Database(path=root)
fd_data = db.get_feature_descriptors(Constants.ResNet_FC_1000)
data_list = [fd_data[i][:10] for i in fd_data]

In [ ]:
from MDS import mds
import matplotlib.pyplot as plt
dim, sa, new_representation = mds.MDS(data_list[:50])
print(dim, sa[-1])
plt.plot(sa)

In [4]:
new_representation

array([[0.3897156 ],
       [0.49437481],
       [0.43442205],
       [0.55414465],
       [0.3058907 ],
       [0.56673387],
       [0.40009966],
       [0.5256387 ],
       [0.3370569 ],
       [0.54199661]])

In [ ]:
dim, sa, new_representation = mds.MDS(data_list, dim=2)
print(dim, sa[-1])
plt.plot(sa)

/Users/gkulk/Documents/Fall 23 Courses/515/Phase 3/multimedia-and-web-databases/MDS/mds.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  * ((X[i, l] - X[j, l]) / d[i, j])
/Users/gkulk/Documents/Fall 23 Courses/515/Phase 3/multimedia-and-web-databases/MDS/mds.py:19: RuntimeWarning: invalid value encountered in scalar multiply
  (delta_ki - delta_kj)
